In [1]:
!pip install pandas
!pip install numpy
!pip install pyarrow
!pip install fastparquet

In [2]:
import datetime
import multiprocessing
from multiprocessing import Pool
import warnings

import numpy as np
import pandas as pd

def set_up_printing():
    """Sets up display parameters"""

    max_dimensions_display_limit = 1000
    pd.set_option('display.max_columns', max_dimensions_display_limit)
    pd.set_option('display.max_colwidth', None)

    pd.set_option('display.max_rows', max_dimensions_display_limit)
    pd.set_option('display.width', None)

    pd.set_option('display.max_info_columns', max_dimensions_display_limit)
    pd.set_option('display.max_info_rows', max_dimensions_display_limit)

    warnings.filterwarnings('ignore')


def get_low_informative(dataframe, limit):
    """Gets low informative columns"""
    low_information_cols = []

    for col in dataframe.columns:
        # наибольшая относительная частота значения признака
        top_freq = dataframe[col].value_counts(normalize=True).max()
        # доля уникальных значений признака
        nunique_ratio = dataframe[col].nunique() / dataframe[col].count()
        # сравниваем с пороговым значением заданным экспертно
        if top_freq > limit:
            print(f'{col}: {round(top_freq * 100, 2)}% одинаковых значений')
            low_information_cols.append(col)
        # сравниваем долю уникальных значений с порогом
        if nunique_ratio > limit:
            print(f'{col}: {round(nunique_ratio * 100, 2)}% уникальных значений')
            low_information_cols.append(col)

    return low_information_cols


def info_aux_and_nans(dataframe, message):
    """Prints dataframe info and shows nan stats"""
    print(message, '\n')
    dataframe.info()
    dataframe.isnull().sum()
    print('\n')


def shape(dataframe, message):
    """Prints dataframe shape"""
    print(message, '\n')
    print(' - размерность набора данных - ', dataframe.shape)
    print('\n')


def f(x):
    x_sum = x.sum()
    x_sum['user_id'] = x.iloc[0, 0]
    return x_sum



# настроим отображение при выводе в консоль
set_up_printing()

print(datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

# считаем данные и одновременно удалим пустые строки
dataframe = pd.read_parquet(r'/mnt/c/Users/M_N_K/kosta_2023_dataton/train_mfti.parquet').head(1000)
shape(dataframe, 'Входной датасет')
get_low_informative(dataframe, 0.99)
info_aux_and_nans(dataframe,
                  'Входной датасет после удаления дубликатов и проверки на информативность')

dataframe = dataframe.drop_duplicates()

# в случае продакшен проекта скорее всего стоило бы рассчитать на всех событиях, предварительно
# отбирая данные на основе требований заказчика в первую очередь, а также разведочного анализа
# в данном случае отбросим все preview значения, что позволит несколько уменьшить размерность
dataframe = dataframe[dataframe['event_type'].isin(
    ['show_vacancy', 'click_contacts', 'click_favorite', 'click_phone', 'click_response'])]
dataframe['user_id'] = dataframe['user_id'].fillna(dataframe['cookie_id'])
dataframe['activity'] = dataframe['vacancy_id_'].astype(str) + '_' + dataframe['event_type'].astype(str)
dataframe = dataframe[['user_id', 'activity']]

print(datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
print('one hot encoding is in progress..')

dataframe = pd.get_dummies(dataframe, columns=['activity'], sparse=True)

27/04/2023 09:42:29
Входной датасет 

 - размерность набора данных -  (1000, 6)


event_date: 100.0% одинаковых значений
Входной датасет после удаления дубликатов и проверки на информативность 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   event_date       1000 non-null   object
 1   event_timestamp  1000 non-null   int64 
 2   vacancy_id_      1000 non-null   int64 
 3   cookie_id        1000 non-null   object
 4   user_id          942 non-null    object
 5   event_type       1000 non-null   object
dtypes: int64(2), object(4)
memory usage: 47.0+ KB


27/04/2023 09:42:32
one hot encoding is in progress..


In [3]:
dataframe.columns.tolist()[:10]

['user_id',
 'activity_100101_show_vacancy',
 'activity_100133_show_vacancy',
 'activity_100411_show_vacancy',
 'activity_100677_show_vacancy',
 'activity_101049_show_vacancy',
 'activity_101147_show_vacancy',
 'activity_101190_show_vacancy',
 'activity_101194_show_vacancy',
 'activity_101228_show_vacancy']

In [4]:
print(datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
print('grouping by user is in progress..')

grouped_by_user = dataframe.groupby('user_id', sort=False)[dataframe.columns.tolist()]

print(datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
print('creation of dataframes from groups is in progress..')

dataframes_by_user = list()
i = 0
for separated_gdf in (group for name, group in grouped_by_user):
    i = i + 1
    if i % 5 == 0:
        print('Separated 5 more gdf: ' + str(i))
    dataframes_by_user.append(separated_gdf)

len(dataframes_by_user)

27/04/2023 09:42:32
grouping by user is in progress..
27/04/2023 09:42:32
creation of dataframes from groups is in progress..
Separated 5 more gdf: 5
Separated 5 more gdf: 10
Separated 5 more gdf: 15
Separated 5 more gdf: 20
Separated 5 more gdf: 25
Separated 5 more gdf: 30
Separated 5 more gdf: 35
Separated 5 more gdf: 40
Separated 5 more gdf: 45
Separated 5 more gdf: 50
Separated 5 more gdf: 55
Separated 5 more gdf: 60
Separated 5 more gdf: 65
Separated 5 more gdf: 70
Separated 5 more gdf: 75
Separated 5 more gdf: 80
Separated 5 more gdf: 85


85

In [5]:
print(datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
print('multiprocessing user vectorization is in progress..')

# ctx = multiprocessing.get_context('spawn')
# pool = ctx.Pool(32)
pool = Pool(64)
results = pool.map(f, dataframes_by_user)
pool.close()
pool.join()

27/04/2023 09:42:33
multiprocessing user vectorization is in progress..


In [6]:
print(datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
print('writing to disc is in progress..')

user_vectors = pd.DataFrame(results)
user_vectors.to_csv(r'/mnt/c/Users/M_N_K/kosta_2023_dataton/user_vectors_parallel_version.to_csv')

print('completed..')
user_vectors.head(3)

27/04/2023 09:42:34
writing to disc is in progress..
completed..


,user_id,activity_100101_show_vacancy,activity_100133_show_vacancy,activity_100411_show_vacancy,activity_100677_show_vacancy,activity_101049_show_vacancy,activity_101147_show_vacancy,activity_101190_show_vacancy,activity_101194_show_vacancy,activity_101228_show_vacancy,activity_101410_show_vacancy,activity_101462_show_vacancy,activity_101670_show_vacancy,activity_101880_show_vacancy,activity_102136_show_vacancy,activity_102279_show_vacancy,activity_102305_show_vacancy,activity_102335_show_vacancy,activity_102358_show_vacancy,activity_102608_show_vacancy,activity_102616_show_vacancy,activity_102914_show_vacancy,activity_102963_show_vacancy,activity_102981_show_vacancy,activity_103098_show_vacancy,activity_103281_show_vacancy,activity_104178_show_vacancy,activity_104549_show_vacancy,activity_105228_show_vacancy,activity_105411_show_vacancy,activity_105575_show_vacancy,activity_105594_show_vacancy,activity_105819_show_vacancy,activity_105826_show_vacancy,activity_106293_show_vacancy,activity_106495_show_vacancy,activity_106724_show_vacancy,activity_107134_show_vacancy,activity_107167_show_vacancy,activity_107223_show_vacancy,activity_107229_show_vacancy,activity_107242_show_vacancy,activity_107342_show_vacancy,activity_107482_show_vacancy,activity_107558_show_vacancy,activity_107617_show_vacancy,activity_107825_show_vacancy,activity_107830_show_vacancy,activity_107882_show_vacancy,activity_108010_show_vacancy,activity_108098_show_vacancy,activity_108242_show_vacancy,activity_108331_show_vacancy,activity_108347_show_vacancy,activity_108889_show_vacancy,activity_108912_show_vacancy,activity_109069_show_vacancy,activity_109632_show_vacancy,activity_109685_show_vacancy,activity_109869_show_vacancy,activity_109978_show_vacancy,activity_110421_show_vacancy,activity_110529_show_vacancy,activity_110740_show_vacancy,activity_110752_show_vacancy,activity_111027_show_vacancy,activity_111039_show_vacancy,activity_111174_show_vacancy,activity_111275_show_vacancy,activity_111505_show_vacancy,activity_111630_show_vacancy,activity_111706_show_vacancy,activity_111769_show_vacancy,activity_111867_show_vacancy,activity_112005_show_vacancy,activity_112251_show_vacancy,activity_112425_show_vacancy,activity_112506_show_vacancy,activity_112578_show_vacancy,activity_112892_show_vacancy,activity_113018_show_vacancy,activity_113217_show_vacancy,activity_113295_show_vacancy,activity_113305_show_vacancy,activity_113494_show_vacancy,activity_113579_show_vacancy,activity_114024_show_vacancy,activity_114041_show_vacancy,activity_114319_show_vacancy,activity_114889_show_vacancy,activity_115414_show_vacancy,activity_115570_show_vacancy,activity_115589_show_vacancy,activity_115924_show_vacancy,activity_115977_show_vacancy,activity_116080_show_vacancy,activity_116222_show_vacancy,activity_116433_show_vacancy,activity_116542_show_vacancy,activity_116552_show_vacancy,activity_116653_show_vacancy,activity_116716_show_vacancy,activity_116900_show_vacancy,activity_116927_show_vacancy,activity_117197_show_vacancy,activity_117496_show_vacancy,activity_117770_show_vacancy,activity_118510_show_vacancy,activity_118598_show_vacancy,activity_118606_show_vacancy,activity_118667_show_vacancy,activity_118829_show_vacancy,activity_118869_show_vacancy,activity_119082_show_vacancy,activity_119095_show_vacancy,activity_119161_show_vacancy,activity_119418_show_vacancy,activity_119437_show_vacancy,activity_119571_show_vacancy,activity_119600_show_vacancy,activity_119643_show_vacancy,activity_119727_show_vacancy,activity_119806_show_vacancy,activity_119824_show_vacancy,activity_119839_show_vacancy,activity_119855_show_vacancy,activity_119890_show_vacancy,activity_119981_show_vacancy,activity_119999_show_vacancy,activity_120075_show_vacancy,activity_120102_show_vacancy,activity_120252_show_vacancy,activity_120269_show_vacancy,activity_120304_show_vacancy,activity_120306_show_vacancy,activity_120329_show_vacancy,activity_120344_show_vacancy,activity_120365_show_vacancy,activity_120426_sh

In [7]:
check_result = pd.read_csv(r'/mnt/c/Users/M_N_K/kosta_2023_dataton/user_vectors_parallel_version.to_csv')
check_result.shape

(85, 500)

In [8]:
check_result.head(10)

,Unnamed: 0,user_id,activity_100101_show_vacancy,activity_100133_show_vacancy,activity_100411_show_vacancy,activity_100677_show_vacancy,activity_101049_show_vacancy,activity_101147_show_vacancy,activity_101190_show_vacancy,activity_101194_show_vacancy,activity_101228_show_vacancy,activity_101410_show_vacancy,activity_101462_show_vacancy,activity_101670_show_vacancy,activity_101880_show_vacancy,activity_102136_show_vacancy,activity_102279_show_vacancy,activity_102305_show_vacancy,activity_102335_show_vacancy,activity_102358_show_vacancy,activity_102608_show_vacancy,activity_102616_show_vacancy,activity_102914_show_vacancy,activity_102963_show_vacancy,activity_102981_show_vacancy,activity_103098_show_vacancy,activity_103281_show_vacancy,activity_104178_show_vacancy,activity_104549_show_vacancy,activity_105228_show_vacancy,activity_105411_show_vacancy,activity_105575_show_vacancy,activity_105594_show_vacancy,activity_105819_show_vacancy,activity_105826_show_vacancy,activity_106293_show_vacancy,activity_106495_show_vacancy,activity_106724_show_vacancy,activity_107134_show_vacancy,activity_107167_show_vacancy,activity_107223_show_vacancy,activity_107229_show_vacancy,activity_107242_show_vacancy,activity_107342_show_vacancy,activity_107482_show_vacancy,activity_107558_show_vacancy,activity_107617_show_vacancy,activity_107825_show_vacancy,activity_107830_show_vacancy,activity_107882_show_vacancy,activity_108010_show_vacancy,activity_108098_show_vacancy,activity_108242_show_vacancy,activity_108331_show_vacancy,activity_108347_show_vacancy,activity_108889_show_vacancy,activity_108912_show_vacancy,activity_109069_show_vacancy,activity_109632_show_vacancy,activity_109685_show_vacancy,activity_109869_show_vacancy,activity_109978_show_vacancy,activity_110421_show_vacancy,activity_110529_show_vacancy,activity_110740_show_vacancy,activity_110752_show_vacancy,activity_111027_show_vacancy,activity_111039_show_vacancy,activity_111174_show_vacancy,activity_111275_show_vacancy,activity_111505_show_vacancy,activity_111630_show_vacancy,activity_111706_show_vacancy,activity_111769_show_vacancy,activity_111867_show_vacancy,activity_112005_show_vacancy,activity_112251_show_vacancy,activity_112425_show_vacancy,activity_112506_show_vacancy,activity_112578_show_vacancy,activity_112892_show_vacancy,activity_113018_show_vacancy,activity_113217_show_vacancy,activity_113295_show_vacancy,activity_113305_show_vacancy,activity_113494_show_vacancy,activity_113579_show_vacancy,activity_114024_show_vacancy,activity_114041_show_vacancy,activity_114319_show_vacancy,activity_114889_show_vacancy,activity_115414_show_vacancy,activity_115570_show_vacancy,activity_115589_show_vacancy,activity_115924_show_vacancy,activity_115977_show_vacancy,activity_116080_show_vacancy,activity_116222_show_vacancy,activity_116433_show_vacancy,activity_116542_show_vacancy,activity_116552_show_vacancy,activity_116653_show_vacancy,activity_116716_show_vacancy,activity_116900_show_vacancy,activity_116927_show_vacancy,activity_117197_show_vacancy,activity_117496_show_vacancy,activity_117770_show_vacancy,activity_118510_show_vacancy,activity_118598_show_vacancy,activity_118606_show_vacancy,activity_118667_show_vacancy,activity_118829_show_vacancy,activity_118869_show_vacancy,activity_119082_show_vacancy,activity_119095_show_vacancy,activity_119161_show_vacancy,activity_119418_show_vacancy,activity_119437_show_vacancy,activity_119571_show_vacancy,activity_119600_show_vacancy,activity_119643_show_vacancy,activity_119727_show_vacancy,activity_119806_show_vacancy,activity_119824_show_vacancy,activity_119839_show_vacancy,activity_119855_show_vacancy,activity_119890_show_vacancy,activity_119981_show_vacancy,activity_119999_show_vacancy,activity_120075_show_vacancy,activity_120102_show_vacancy,activity_120252_show_vacancy,activity_120269_show_vacancy,activity_120304_show_vacancy,activity_120306_show_vacancy,activity_120329_show_vacancy,activity_120344_show_vacancy,activity_120365_show_vacancy,activit